In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate() 

In [ ]:
using CSV
using DataFrames
df=CSV.read("data/natsal_1y_18to44.csv", DataFrame);

In [ ]:
MM_all=df[(1 .+4 .*(0:2)),3:end]|>Matrix|>permutedims
HM_all=df[(2 .+4 .*(0:2)),3:end]|>Matrix|>permutedims;
HW_all=df[(4 .+4 .*(0:2)),3:end]|>Matrix|>permutedims;

In [ ]:
params_main = (
    κ_msm=0.77, α_msm=0.10,  # 2000–2020
    κ_hem=1.68, α_hem=0.01, # 2000–2010
    κ_hew=2.14, α_hew=0.01 # 2010
)

In [ ]:
module Baseline
κ_msm, α_msm, κ_hem, α_hem, κ_hew, α_hew = 
    getindex.(Ref(Main.params_main), [:κ_msm, :α_msm, :κ_hem, :α_hem, :κ_hew, :α_hew])

printtable=true
outbreak_iters=100000
partnersdata=[sum(Main.MM_all,dims=2) sum(Main.HM_all[:,1:2],dims=2) sum(Main.HW_all[:,2:2],dims=2)]
hist_upp=0.08
include("src/outputs.jl")
end

In [ ]:
module UKonly
κ_msm, α_msm, κ_hem, α_hem, κ_hew, α_hew = 
    getindex.(Ref(Main.params_main), [:κ_msm, :α_msm, :κ_hem, :α_hem, :κ_hew, :α_hew])
currentcases = 190 # as of 31 May in UK
printtable=true
outbreak_iters=100000
partnersdata=[sum(Main.MM_all,dims=2) sum(Main.HM_all[:,1:2],dims=2) sum(Main.HW_all[:,2:2],dims=2)]
include("src/outputs.jl")
end

In [ ]:
module InfectiousPeriod
κ_msm, α_msm, κ_hem, α_hem, κ_hew, α_hew = 
    getindex.(Ref(Main.params_main), [:κ_msm, :α_msm, :κ_hem, :α_hem, :κ_hew, :α_hew])
R0only = true
R0list = Vector{Matrix{Float64}}(undef,0)
inf_period=0
for inf_p in [7,14,28]
    global inf_period=inf_p
    include("src/outputs.jl")
    push!(R0list,R0values)
end
R0matrix=reduce(hcat,R0list)[:,vec(reshape(1:6,2,3)')]

R0valuesplot=plot(sarrange,R0matrix,
yaxis=:log,ylim=(0.01,100),linewidth=1.5, palette=:balance,color=[3:-1:1;4:6]'.*35,
xlab="SAR per sexually-associated contact", ylab="R₀ over sexual network",legend=:bottomright,
label=string.([7 14 28]).*[" d"])
hline!(R0valuesplot,[1],linestyle=:dot,color=:black,label="") |> display

R0redplot=plot(sarrange,max.(0,min.(1,1 .- 1 ./R0matrix)),
ylim=(0,1),linewidth=1.5, palette=:balance,color=[3:-1:1;4:6]'.*35,label=string.([7 14 28]).*[" d"],
xlab="SAR per sexually-associated contact", ylab="relative reduction in R₀ for control",legend=:bottomright)|>display
end

In [ ]:
params_00_10 = (
    κ_msm=0.71, α_msm=0.13, 
    κ_hem=1.68,α_hem=0.01,
    κ_hew=2.33,α_hew=BigFloat(0.003)) 
params_20 = (
    κ_msm=0.83, α_msm=0.09, 
    κ_hem=2.12,α_hem=0.01,
    κ_hew=2.55,α_hew=0.01) 

In [ ]:
module NatsalSubsets
R0only = true
R0list = Vector{Matrix{Float64}}(undef,0)
inf_period=21
paramset=[Main.params_00_10, Main.params_20]
κ_msm, α_msm, κ_hem, α_hem, κ_hew, α_hew = zeros(6)
for params in paramset
    global κ_msm, α_msm, κ_hem, α_hem, κ_hew, α_hew = 
        getindex.(Ref(params), [:κ_msm, :α_msm, :κ_hem, :α_hem, :κ_hew, :α_hew]).|>BigFloat
    include("src/outputs.jl")
    push!(R0list,R0values)
end
R0matrix=reduce(hcat,R0list)[:,vec(reshape(1:4,2,2)')]

R0valuesplot=plot(sarrange,R0matrix,
yaxis=:log,ylim=(0.001,100),linewidth=1.5, palette=:balance,color=[1;2;6;5]'.*35,
xlab="SAR per sexually-associated contact", ylab="R₀ over sexual network",legend=:bottomright,
label=["2000-2010" "2020"])
hline!(R0valuesplot,[1],linestyle=:dot,color=:black,label="") |> display

R0redplot=plot(sarrange,max.(0,min.(1,1 .- 1 ./R0matrix)),
ylim=(0,1),linewidth=1.5, palette=:balance,color=[1;2;6;5]'.*35,label=["2000-2010" "2020"],
xlab="SAR per sexually-associated contact", ylab="relative reduction in R₀ for control",legend=:bottomright)|>display
end